# 3. 데이터 전처리 - 재난대응 시스템

재난대응 시스템 데이터로 치환하여 전처리합니다.
- Neo4j에 적재하기 위한 노드/관계 구조 준비
- GeoPandas로 공간 데이터 처리 (Point geometry)
- 좌표 변환 및 정규화 (WGS84)


In [1]:
# 필요한 라이브러리 import
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from typing import Dict, List, Optional
import os
import warnings
import glob
import re
from pathlib import Path
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")


라이브러리 로드 완료


## 1. 데이터 로드 및 기본 정보 확인


In [2]:
# 데이터 파일 경로
DATA_DIR = "data"
OUTPUT_DIR = "data/processed"

# 출력 디렉토리 생성
os.makedirs(OUTPUT_DIR, exist_ok=True)

# CSV 파일 경로
files = {
    'outdoor_shelter': f'{DATA_DIR}/서울시 지진옥외대피소.csv',
    'indoor_shelter': f'{DATA_DIR}/서울시 지진대피소 현황.csv',
    'temporary_housing': f'{DATA_DIR}/서울시 이재민임시주거시설(지진겸용).csv',
    'water_facility': f'{DATA_DIR}/서울시 민방위급수시설 인허가 정보.csv'
}

# 데이터 로드 (cp949 인코딩)
data = {}
for name, path in files.items():
    try:
        df = pd.read_csv(path, encoding='cp949', low_memory=False)
        data[name] = df
        print(f"\n{name}: {len(df)} rows, {len(df.columns)} columns")
        print(f"컬럼: {list(df.columns)[:5]}")
    except Exception as e:
        print(f"{name} 로드 실패: {e}")

print("\n데이터 로드 완료")



outdoor_shelter: 1585 rows, 14 columns
컬럼: ['시설번호', '지역코드', '시설일련번호', '시도명', '시군구명']

indoor_shelter: 1551 rows, 11 columns
컬럼: ['피난처 ID', '시도명', '시군구명', '시설명', '상세주소']

temporary_housing: 821 rows, 15 columns
컬럼: ['시설번호', '지역코드', '시설일련번호', '시도명', '시군구명']

water_facility: 2611 rows, 29 columns
컬럼: ['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드']

데이터 로드 완료


## 2. 옥외대피소 데이터 전처리


In [3]:
# 옥외대피소 데이터 정규화
if 'outdoor_shelter' in data:
    df_outdoor = data['outdoor_shelter'].copy()
    
    # 컬럼명 확인
    print("옥외대피소 컬럼:", df_outdoor.columns.tolist())
    
    # 데이터 정규화
    df_outdoor_clean = pd.DataFrame()
    
    # 기본 정보
    df_outdoor_clean['shelter_id'] = df_outdoor['시설번호'].astype(str)
    df_outdoor_clean['region_code'] = df_outdoor['지역코드'].astype(str)
    df_outdoor_clean['shelter_name'] = df_outdoor['수용시설명'].fillna('').astype(str)
    df_outdoor_clean['sido'] = df_outdoor['시도명'].fillna('').astype(str)
    df_outdoor_clean['sigungu'] = df_outdoor['시군구명'].fillna('').astype(str)
    df_outdoor_clean['address'] = df_outdoor['상세주소'].fillna('').astype(str)
    
    # 좌표 처리
    if '경도' in df_outdoor.columns and '위도' in df_outdoor.columns:
        df_outdoor_clean['lon'] = pd.to_numeric(df_outdoor['경도'], errors='coerce')
        df_outdoor_clean['lat'] = pd.to_numeric(df_outdoor['위도'], errors='coerce')
    elif 'X좌표' in df_outdoor.columns and 'Y좌표' in df_outdoor.columns:
        # X, Y 좌표는 보통 TM 좌표계이므로 변환이 필요하지만
        # 여기서는 경도/위도 컬럼이 없으면 X/Y를 그대로 사용 (실제로는 pyproj로 변환 필요)
        df_outdoor_clean['lon'] = pd.to_numeric(df_outdoor['X좌표'], errors='coerce') / 1000  # 대략적 변환
        df_outdoor_clean['lat'] = pd.to_numeric(df_outdoor['Y좌표'], errors='coerce') / 1000
    
    # 시설 유형
    df_outdoor_clean['shelter_type'] = '옥외대피소'
    
    # 시설 면적
    if '시설면적' in df_outdoor.columns:
        df_outdoor_clean['area'] = pd.to_numeric(df_outdoor['시설면적'], errors='coerce')
    
    # 유효한 좌표만 필터링
    df_outdoor_clean = df_outdoor_clean[
        df_outdoor_clean['lon'].notna() & 
        df_outdoor_clean['lat'].notna() &
        (df_outdoor_clean['lon'] > 120) &  # 서울 경도 범위
        (df_outdoor_clean['lon'] < 130) &
        (df_outdoor_clean['lat'] > 35) &    # 서울 위도 범위
        (df_outdoor_clean['lat'] < 40)
    ].copy()
    
    print(f"옥외대피소 전처리 완료: {len(df_outdoor_clean)} 행")
    print(f"좌표 범위: 경도 {df_outdoor_clean['lon'].min():.4f}~{df_outdoor_clean['lon'].max():.4f}, "
          f"위도 {df_outdoor_clean['lat'].min():.4f}~{df_outdoor_clean['lat'].max():.4f}")
    
    # 전처리된 데이터 저장
    data['outdoor_shelter_clean'] = df_outdoor_clean


옥외대피소 컬럼: ['시설번호', '지역코드', '시설일련번호', '시도명', '시군구명', '수용시설명', '법정동코드', '행정동코드', '상세주소', '시설면적', '경도', '위도', 'X좌표', 'Y좌표']
옥외대피소 전처리 완료: 1585 행
좌표 범위: 경도 126.7989~127.1774, 위도 37.4348~37.6889


## 8. 재난 행동요령 문서 데이터 전처리


In [4]:
# docs 폴더의 마크다운 문서들을 로드하고 전처리
DOCS_DIR = "docs"
DOCS_OUTPUT_DIR = "data/processed/docs"

# 출력 디렉토리 생성
os.makedirs(DOCS_OUTPUT_DIR, exist_ok=True)

# 마크다운 파일 목록
md_files = glob.glob(f"{DOCS_DIR}/*.md")
print(f"\n발견된 문서 파일: {len(md_files)} 개")

# 문서 데이터를 저장할 리스트
documents_data = []

for md_file in md_files:
    file_name = Path(md_file).stem
    
    try:
        # 파일 읽기 (UTF-8 인코딩 시도)
        with open(md_file, 'r', encoding='utf-8') as f:
            content = f.read()
        
        # 기본 메타데이터 추출
        # 파일명에서 재난 유형 추출
        disaster_type = "기타"
        if "지진" in file_name:
            disaster_type = "지진"
        elif "공습" in file_name:
            disaster_type = "공습"
        elif "화재" in file_name:
            disaster_type = "화재"
        elif "홍수" in file_name:
            disaster_type = "홍수"
        elif "댐붕괴" in file_name:
            disaster_type = "댐붕괴"
        elif "전기" in file_name or "가스" in file_name:
            disaster_type = "전기·가스사고"
        elif "정전" in file_name or "전력" in file_name:
            disaster_type = "정전·전력부족"
        elif "폭발" in file_name:
            disaster_type = "폭발사고"
        elif "철도" in file_name or "지하철" in file_name:
            disaster_type = "철도·지하철사고"
        elif "터널" in file_name:
            disaster_type = "터널사고"
        elif "사업장" in file_name:
            disaster_type = "사업장사고"
        elif "테러" in file_name:
            disaster_type = "테러"
        elif "자연재난" in file_name:
            disaster_type = "자연재난"
        elif "사회재난" in file_name:
            disaster_type = "사회재난"
        elif "산행" in file_name:
            disaster_type = "산행안전"
        
        # 문서를 섹션별로 분할 (## 제목 기준)
        sections = []
        lines = content.split('\n')
        current_section = []
        current_title = ""
        
        for line in lines:
            if line.startswith('##'):
                if current_section:
                    sections.append({
                        'title': current_title,
                        'content': '\n'.join(current_section).strip()
                    })
                current_title = line.replace('##', '').strip()
                current_section = []
            else:
                current_section.append(line)
        
        # 마지막 섹션 추가
        if current_section:
            sections.append({
                'title': current_title if current_title else "전체",
                'content': '\n'.join(current_section).strip()
            })
        
        # 문서 데이터 저장
        for idx, section in enumerate(sections):
            doc_id = f"doc_{file_name}_{idx}"
            documents_data.append({
                'doc_id': doc_id,
                'file_name': file_name,
                'file_path': md_file,
                'disaster_type': disaster_type,
                'section_title': section['title'],
                'section_index': idx,
                'content': section['content'],
                'full_content': content,
                'content_length': len(section['content'])
            })
        
        print(f"  - {file_name}: {len(sections)} 섹션, {len(content)} 문자")
        
    except Exception as e:
        print(f"  - {file_name} 로드 실패: {e}")

# 문서 데이터를 DataFrame으로 변환
if documents_data:
    docs_df = pd.DataFrame(documents_data)
    print(f"\n총 문서 섹션: {len(docs_df)} 개")
    print(f"재난 유형별 통계:")
    print(docs_df['disaster_type'].value_counts())
    
    # CSV로 저장 (RAG/Chroma 적재용)
    docs_df.to_csv(
        f'{DOCS_OUTPUT_DIR}/disaster_guidelines_sections.csv',
        index=False,
        encoding='utf-8-sig'
    )
    print(f"\n문서 데이터 저장: {DOCS_OUTPUT_DIR}/disaster_guidelines_sections.csv")
    
    # 전체 문서 내용도 별도로 저장 (Chroma용)
    docs_full_df = docs_df[['doc_id', 'file_name', 'disaster_type', 'section_title', 'content']].copy()
    docs_full_df.to_csv(
        f'{DOCS_OUTPUT_DIR}/disaster_guidelines_for_rag.csv',
        index=False,
        encoding='utf-8-sig'
    )
    print(f"RAG용 문서 데이터 저장: {DOCS_OUTPUT_DIR}/disaster_guidelines_for_rag.csv")
else:
    print("\n문서 데이터가 없습니다.")
    docs_df = pd.DataFrame()



발견된 문서 파일: 16 개
  - 철도·지하철 사고 국민행동요령: 50 섹션, 6005 문자
  - 폭발사고 국민행동요령: 38 섹션, 5177 문자
  - 자연재난 행동요령 통합 가이드: 289 섹션, 19988 문자
  - 도로 터널사고 국민행동요령: 33 섹션, 4236 문자
  - 비상시 국민행동요령: 116 섹션, 8770 문자
  - 산행안전행동요령_산행안전사고: 5 섹션, 1369 문자
  - 정전 및 전력부족 국민행동요령: 37 섹션, 4957 문자
  - 홍수행동요령: 21 섹션, 2049 문자
  - 사회재난 행동요령 통합 가이드: 88 섹션, 11692 문자
  - 사업장 대규모 인적사고 국민행동요령 : 43 섹션, 7587 문자
  - 공습행동요령: 26 섹션, 2549 문자
  - 테러 대응 국민행동요령: 19 섹션, 2254 문자
  - 댐붕괴 국민행동요령: 35 섹션, 4381 문자
  - 화재행동요령: 30 섹션, 2595 문자
  - 전기·가스사고 국민행동요령: 74 섹션, 7191 문자
  - 지진행동요령: 29 섹션, 2634 문자

총 문서 섹션: 933 개
재난 유형별 통계:
disaster_type
자연재난        289
기타          121
사회재난         88
전기·가스사고      74
철도·지하철사고     50
사업장사고        43
폭발사고         38
정전·전력부족      37
댐붕괴          35
터널사고         33
화재           30
지진           29
공습           26
홍수           21
테러           19
Name: count, dtype: int64

문서 데이터 저장: data/processed/docs/disaster_guidelines_sections.csv
RAG용 문서 데이터 저장: data/processed/docs/disaster_guidelin

In [5]:
# 실내대피소 데이터 정규화
if 'indoor_shelter' in data:
    df_indoor = data['indoor_shelter'].copy()
    
    # 데이터 정규화
    df_indoor_clean = pd.DataFrame()
    
    # 기본 정보
    df_indoor_clean['shelter_id'] = df_indoor['피난처 ID'].astype(str)
    df_indoor_clean['shelter_name'] = df_indoor['시설명'].fillna('').astype(str)
    df_indoor_clean['sido'] = df_indoor['시도명'].fillna('').astype(str)
    df_indoor_clean['sigungu'] = df_indoor['시군구명'].fillna('').astype(str)
    df_indoor_clean['address'] = df_indoor['상세주소'].fillna('').astype(str)
    
    # 좌표 처리
    if '경도' in df_indoor.columns and '위도' in df_indoor.columns:
        df_indoor_clean['lon'] = pd.to_numeric(df_indoor['경도'], errors='coerce')
        df_indoor_clean['lat'] = pd.to_numeric(df_indoor['위도'], errors='coerce')
    
    # 시설 유형
    df_indoor_clean['shelter_type'] = '실내대피소'
    
    # 시설 면적
    if '시설면적' in df_indoor.columns:
        df_indoor_clean['area'] = pd.to_numeric(df_indoor['시설면적'], errors='coerce')
    
    # 구분 정보
    if '구분명' in df_indoor.columns:
        df_indoor_clean['category'] = df_indoor['구분명'].fillna('').astype(str)
    
    # 유효한 좌표만 필터링
    df_indoor_clean = df_indoor_clean[
        df_indoor_clean['lon'].notna() & 
        df_indoor_clean['lat'].notna() &
        (df_indoor_clean['lon'] > 120) &
        (df_indoor_clean['lon'] < 130) &
        (df_indoor_clean['lat'] > 35) &
        (df_indoor_clean['lat'] < 40)
    ].copy()
    
    print(f"실내대피소 전처리 완료: {len(df_indoor_clean)} 행")
    
    # 전처리된 데이터 저장
    data['indoor_shelter_clean'] = df_indoor_clean


실내대피소 전처리 완료: 1551 행


In [6]:
# 재해위험지구 데이터 전처리
if 'risk_zone' in data:
    df_risk = data['risk_zone'].copy()
    
    df_risk_clean = pd.DataFrame()
    
    # 기본 정보
    if 'DST_RSK_DSTRCT_MNG_NO' in df_risk.columns:
        df_risk_clean['risk_id'] = df_risk['DST_RSK_DSTRCT_MNG_NO'].astype(str)
    if 'DST_RSK_DSTRCT_NM' in df_risk.columns:
        df_risk_clean['name'] = df_risk['DST_RSK_DSTRCT_NM'].fillna('').astype(str)
    if 'DST_RSK_DSTRCT_GRD_CD' in df_risk.columns:
        df_risk_clean['grade'] = df_risk['DST_RSK_DSTRCT_GRD_CD'].fillna('').astype(str)
    if 'DST_RSK_DSTRCT_TYPE_CD' in df_risk.columns:
        df_risk_clean['risk_type'] = df_risk['DST_RSK_DSTRCT_TYPE_CD'].fillna('').astype(str)
    
    # 주소에서 시도/시군구 추출 시도
    if 'DADDR' in df_risk.columns or 'RONA_DADDR' in df_risk.columns:
        address_col = 'DADDR' if 'DADDR' in df_risk.columns else 'RONA_DADDR'
        addresses = df_risk[address_col].fillna('').astype(str)
        # 주소에서 시도/시군구 추출 (예: "서울특별시 강남구 ...")
        df_risk_clean['address'] = addresses
        df_risk_clean['sido'] = addresses.str.split().str[0].fillna('')
        df_risk_clean['sigungu'] = addresses.str.split().str[1].fillna('') if len(addresses.str.split()) > 1 else ''
    
    # 좌표 정보 (있는 경우)
    if 'XCRD' in df_risk.columns and 'YCRD' in df_risk.columns:
        # TM 좌표계인 경우 변환 필요하지만 일단 그대로 저장
        df_risk_clean['lon'] = pd.to_numeric(df_risk['XCRD'], errors='coerce')
        df_risk_clean['lat'] = pd.to_numeric(df_risk['YCRD'], errors='coerce')
    
    df_risk_clean['risk_category'] = '재해위험지구'
    df_risk_clean = df_risk_clean[df_risk_clean['risk_id'].notna()].copy()
    
    data['risk_zone_clean'] = df_risk_clean
    print(f"재해위험지구 전처리 완료: {len(df_risk_clean)} 행")

# 인명피해우려지역 데이터 전처리
if 'casualty_risk' in data:
    df_casualty = data['casualty_risk'].copy()
    
    df_casualty_clean = pd.DataFrame()
    
    # 기본 정보
    if 'HULI_DAM_CNCR_RGN_MNG_NO' in df_casualty.columns:
        df_casualty_clean['risk_id'] = df_casualty['HULI_DAM_CNCR_RGN_MNG_NO'].astype(str)
    if 'DSTRCT_NM' in df_casualty.columns:
        df_casualty_clean['name'] = df_casualty['DSTRCT_NM'].fillna('').astype(str)
    if 'DAM_CNCR_RGN_TYPE_CD' in df_casualty.columns:
        df_casualty_clean['risk_type'] = df_casualty['DAM_CNCR_RGN_TYPE_CD'].fillna('').astype(str)
    
    # 주소 정보
    if 'DADDR' in df_casualty.columns or 'RONA_DADDR' in df_casualty.columns:
        address_col = 'DADDR' if 'DADDR' in df_casualty.columns else 'RONA_DADDR'
        addresses = df_casualty[address_col].fillna('').astype(str)
        df_casualty_clean['address'] = addresses
        df_casualty_clean['sido'] = addresses.str.split().str[0].fillna('')
        df_casualty_clean['sigungu'] = addresses.str.split().str[1].fillna('') if len(addresses.str.split()) > 1 else ''
    
    # 좌표 정보
    if 'LAT' in df_casualty.columns and 'LOT' in df_casualty.columns:
        df_casualty_clean['lat'] = pd.to_numeric(df_casualty['LAT'], errors='coerce')
        df_casualty_clean['lon'] = pd.to_numeric(df_casualty['LOT'], errors='coerce')
    
    # 예상 피해 정보
    if 'EXPC_DAM_NOPE' in df_casualty.columns:
        df_casualty_clean['expected_casualties'] = pd.to_numeric(df_casualty['EXPC_DAM_NOPE'], errors='coerce')
    
    df_casualty_clean['risk_category'] = '인명피해우려지역'
    df_casualty_clean = df_casualty_clean[df_casualty_clean['risk_id'].notna()].copy()
    
    # 유효한 좌표만 필터링 (서울 지역만)
    if 'lon' in df_casualty_clean.columns and 'lat' in df_casualty_clean.columns:
        df_casualty_clean = df_casualty_clean[
            df_casualty_clean['lon'].notna() & 
            df_casualty_clean['lat'].notna() &
            (df_casualty_clean['lon'] > 120) & (df_casualty_clean['lon'] < 130) &
            (df_casualty_clean['lat'] > 35) & (df_casualty_clean['lat'] < 40)
        ].copy()
    
    data['casualty_risk_clean'] = df_casualty_clean
    print(f"인명피해우려지역 전처리 완료: {len(df_casualty_clean)} 행")

# 산사태우려지역 데이터 전처리
if 'landslide_risk' in data:
    df_landslide = data['landslide_risk'].copy()
    
    df_landslide_clean = pd.DataFrame()
    
    if 'LNLD_RSK_MNG_NO' in df_landslide.columns:
        df_landslide_clean['risk_id'] = df_landslide['LNLD_RSK_MNG_NO'].astype(str)
    if 'DSTRCT_NM' in df_landslide.columns:
        df_landslide_clean['name'] = df_landslide['DSTRCT_NM'].fillna('').astype(str)
    
    if 'DADDR' in df_landslide.columns:
        addresses = df_landslide['DADDR'].fillna('').astype(str)
        df_landslide_clean['address'] = addresses
        df_landslide_clean['sido'] = addresses.str.split().str[0].fillna('')
        df_landslide_clean['sigungu'] = addresses.str.split().str[1].fillna('') if len(addresses.str.split()) > 1 else ''
    
    df_landslide_clean['risk_category'] = '산사태우려지역'
    df_landslide_clean = df_landslide_clean[df_landslide_clean['risk_id'].notna()].copy()
    
    data['landslide_risk_clean'] = df_landslide_clean
    print(f"산사태우려지역 전처리 완료: {len(df_landslide_clean)} 행")

# 붕괴위험지역 데이터 전처리
if 'collapse_risk' in data:
    df_collapse = data['collapse_risk'].copy()
    
    df_collapse_clean = pd.DataFrame()
    
    if 'MNG_NO' in df_collapse.columns:
        df_collapse_clean['risk_id'] = df_collapse['MNG_NO'].astype(str)
    if 'CLPS_RSK_DSTRCT_NM' in df_collapse.columns:
        df_collapse_clean['name'] = df_collapse['CLPS_RSK_DSTRCT_NM'].fillna('').astype(str)
    
    df_collapse_clean['risk_category'] = '붕괴위험지역'
    df_collapse_clean = df_collapse_clean[df_collapse_clean['risk_id'].notna()].copy()
    
    data['collapse_risk_clean'] = df_collapse_clean
    print(f"붕괴위험지역 전처리 완료: {len(df_collapse_clean)} 행")

# 노후위험시설물 데이터 전처리
if 'old_facility' in data:
    df_old = data['old_facility'].copy()
    
    df_old_clean = pd.DataFrame()
    
    if 'DST_RSK_DSTRCT_FCLTY_CD' in df_old.columns:
        df_old_clean['facility_id'] = df_old['DST_RSK_DSTRCT_FCLTY_CD'].astype(str)
    if 'MEMENT_NM' in df_old.columns:
        df_old_clean['name'] = df_old['MEMENT_NM'].fillna('').astype(str)
    
    # 좌표 정보
    if 'LAT' in df_old.columns and 'LOT' in df_old.columns:
        df_old_clean['lat'] = pd.to_numeric(df_old['LAT'], errors='coerce')
        df_old_clean['lon'] = pd.to_numeric(df_old['LOT'], errors='coerce')
    
    # 경보 정보
    if 'WRN_CD' in df_old.columns:
        df_old_clean['warning_level'] = df_old['WRN_CD'].fillna('').astype(str)
    
    df_old_clean['facility_type'] = '노후위험시설물'
    
    # 유효한 좌표만 필터링 (서울 지역만)
    if 'lon' in df_old_clean.columns and 'lat' in df_old_clean.columns:
        df_old_clean = df_old_clean[
            df_old_clean['lon'].notna() & 
            df_old_clean['lat'].notna() &
            (df_old_clean['lon'] > 120) & (df_old_clean['lon'] < 130) &
            (df_old_clean['lat'] > 35) & (df_old_clean['lat'] < 40)
        ].copy()
    
    data['old_facility_clean'] = df_old_clean
    print(f"노후위험시설물 전처리 완료: {len(df_old_clean)} 행")


## 4. 임시주거시설 및 급수시설 데이터 전처리


In [7]:
# 임시주거시설 데이터 정규화
if 'temporary_housing' in data:
    df_housing = data['temporary_housing'].copy()
    
    df_housing_clean = pd.DataFrame()
    df_housing_clean['facility_id'] = df_housing['시설번호'].astype(str)
    df_housing_clean['facility_name'] = df_housing['수용시설명'].fillna('').astype(str)
    df_housing_clean['sido'] = df_housing['시도명'].fillna('').astype(str)
    df_housing_clean['sigungu'] = df_housing['시군구명'].fillna('').astype(str)
    df_housing_clean['address'] = df_housing['상세주소'].fillna('').astype(str)
    
    # 좌표 처리 (경도/위도 또는 X/Y 좌표)
    if '경도' in df_housing.columns and '위도' in df_housing.columns:
        df_housing_clean['lon'] = pd.to_numeric(df_housing['경도'], errors='coerce')
        df_housing_clean['lat'] = pd.to_numeric(df_housing['위도'], errors='coerce')
    elif 'X좌표' in df_housing.columns and 'Y좌표' in df_housing.columns:
        df_housing_clean['lon'] = pd.to_numeric(df_housing['X좌표'], errors='coerce') / 1000
        df_housing_clean['lat'] = pd.to_numeric(df_housing['Y좌표'], errors='coerce') / 1000
    
    df_housing_clean['facility_type'] = '임시주거시설'
    
    if '시설면적' in df_housing.columns:
        df_housing_clean['area'] = pd.to_numeric(df_housing['시설면적'], errors='coerce')
    
    # 유효한 좌표만 필터링
    if 'lon' in df_housing_clean.columns and 'lat' in df_housing_clean.columns:
        df_housing_clean = df_housing_clean[
            df_housing_clean['lon'].notna() & 
            df_housing_clean['lat'].notna() &
            (df_housing_clean['lon'] > 120) &
            (df_housing_clean['lon'] < 130) &
            (df_housing_clean['lat'] > 35) &
            (df_housing_clean['lat'] < 40)
        ].copy()
    
    data['temporary_housing_clean'] = df_housing_clean
    print(f"임시주거시설 전처리 완료: {len(df_housing_clean)} 행")

# 급수시설 데이터 정규화
if 'water_facility' in data:
    df_water = data['water_facility'].copy()
    
    df_water_clean = pd.DataFrame()
    df_water_clean['facility_id'] = df_water['관리번호'].astype(str)
    
    # 주소 정보 추출
    if '소재지전체주소' in df_water.columns:
        address_col = '소재지전체주소'
    elif '도로명전체주소' in df_water.columns:
        address_col = '도로명전체주소'
    else:
        address_col = None
    
    if address_col:
        df_water_clean['address'] = df_water[address_col].fillna('').astype(str)
        df_water_clean['sido'] = df_water_clean['address'].str.split(' ').str[0].fillna('')
        df_water_clean['sigungu'] = df_water_clean['address'].str.split(' ').str[1].fillna('')
    
    # 시설명
    if '사업장명' in df_water.columns:
        df_water_clean['facility_name'] = df_water['사업장명'].fillna('').astype(str)
    
    # 좌표 처리
    if '좌표정보(X)' in df_water.columns and '좌표정보(Y)' in df_water.columns:
        df_water_clean['lon'] = pd.to_numeric(df_water['좌표정보(X)'], errors='coerce')
        df_water_clean['lat'] = pd.to_numeric(df_water['좌표정보(Y)'], errors='coerce')
    elif '경도' in df_water.columns and '위도' in df_water.columns:
        df_water_clean['lon'] = pd.to_numeric(df_water['경도'], errors='coerce')
        df_water_clean['lat'] = pd.to_numeric(df_water['위도'], errors='coerce')
    
    df_water_clean['facility_type'] = '급수시설'
    
    if '영업상태명' in df_water.columns:
        df_water_clean['status'] = df_water['영업상태명'].fillna('').astype(str)
    
    # 유효한 좌표만 필터링 (좌표가 있는 경우만)
    if 'lon' in df_water_clean.columns and 'lat' in df_water_clean.columns:
        df_water_clean = df_water_clean[
            df_water_clean['lon'].notna() & 
            df_water_clean['lat'].notna() &
            (df_water_clean['lon'] > 120) &
            (df_water_clean['lon'] < 130) &
            (df_water_clean['lat'] > 35) &
            (df_water_clean['lat'] < 40)
        ].copy()
    else:
        # 좌표가 없으면 주소만 있는 데이터도 포함
        df_water_clean = df_water_clean[df_water_clean['address'] != ''].copy()
    
    data['water_facility_clean'] = df_water_clean
    print(f"급수시설 전처리 완료: {len(df_water_clean)} 행")


임시주거시설 전처리 완료: 821 행
급수시설 전처리 완료: 0 행


In [8]:
# 각 데이터셋을 GeoDataFrame으로 변환 (WGS84 좌표계)
geo_data = {}

# 옥외대피소
if 'outdoor_shelter_clean' in data:
    df = data['outdoor_shelter_clean'].copy()
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf_outdoor = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    geo_data['outdoor_shelter'] = gdf_outdoor
    print(f"옥외대피소 GeoDataFrame: {len(gdf_outdoor)} 행")

# 실내대피소
if 'indoor_shelter_clean' in data:
    df = data['indoor_shelter_clean'].copy()
    geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
    gdf_indoor = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    geo_data['indoor_shelter'] = gdf_indoor
    print(f"실내대피소 GeoDataFrame: {len(gdf_indoor)} 행")

# 임시주거시설
if 'temporary_housing_clean' in data:
    df = data['temporary_housing_clean'].copy()
    if 'lon' in df.columns and 'lat' in df.columns:
        geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
        gdf_housing = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
        geo_data['temporary_housing'] = gdf_housing
        print(f"임시주거시설 GeoDataFrame: {len(gdf_housing)} 행")

# 급수시설
if 'water_facility_clean' in data:
    df = data['water_facility_clean'].copy()
    if 'lon' in df.columns and 'lat' in df.columns:
        df_coords = df[df['lon'].notna() & df['lat'].notna()].copy()
        if len(df_coords) > 0:
            geometry = [Point(xy) for xy in zip(df_coords['lon'], df_coords['lat'])]
            gdf_water = gpd.GeoDataFrame(df_coords, geometry=geometry, crs='EPSG:4326')
            geo_data['water_facility'] = gdf_water
            print(f"급수시설 GeoDataFrame: {len(gdf_water)} 행")

print("\nGeoDataFrame 생성 완료 (CRS: EPSG:4326)")


옥외대피소 GeoDataFrame: 1585 행
실내대피소 GeoDataFrame: 1551 행
임시주거시설 GeoDataFrame: 821 행

GeoDataFrame 생성 완료 (CRS: EPSG:4326)


## 6. Neo4j 적재용 노드/관계 데이터 준비


In [9]:
# Neo4j에 적재하기 위한 노드와 관계 데이터 준비

# 1. Shelter 노드 (옥외 + 실내 대피소 통합)
shelter_nodes = []

if 'outdoor_shelter_clean' in data:
    for _, row in data['outdoor_shelter_clean'].iterrows():
        shelter_nodes.append({
            'id': f"shelter_{row['shelter_id']}",
            'type': 'Shelter',
            'shelter_id': str(row['shelter_id']),
            'name': str(row['shelter_name']),
            'shelter_type': row['shelter_type'],
            'sido': str(row['sido']),
            'sigungu': str(row['sigungu']),
            'address': str(row['address']),
            'lat': float(row['lat']),
            'lon': float(row['lon']),
            'area': float(row.get('area', 0)) if pd.notna(row.get('area')) else None
        })

if 'indoor_shelter_clean' in data:
    for _, row in data['indoor_shelter_clean'].iterrows():
        shelter_nodes.append({
            'id': f"shelter_{row['shelter_id']}",
            'type': 'Shelter',
            'shelter_id': str(row['shelter_id']),
            'name': str(row['shelter_name']),
            'shelter_type': row['shelter_type'],
            'sido': str(row['sido']),
            'sigungu': str(row['sigungu']),
            'address': str(row['address']),
            'lat': float(row['lat']),
            'lon': float(row['lon']),
            'area': float(row.get('area', 0)) if pd.notna(row.get('area')) else None,
            'category': str(row.get('category', ''))
        })

print(f"Shelter 노드: {len(shelter_nodes)} 개")

# 2. TemporaryHousing 노드
housing_nodes = []
if 'temporary_housing_clean' in data:
    for _, row in data['temporary_housing_clean'].iterrows():
        if pd.notna(row.get('lon')) and pd.notna(row.get('lat')):
            housing_nodes.append({
                'id': f"housing_{row['facility_id']}",
                'type': 'TemporaryHousing',
                'facility_id': str(row['facility_id']),
                'name': str(row['facility_name']),
                'sido': str(row['sido']),
                'sigungu': str(row['sigungu']),
                'address': str(row['address']),
                'lat': float(row['lat']),
                'lon': float(row['lon']),
                'area': float(row.get('area', 0)) if pd.notna(row.get('area')) else None
            })

print(f"TemporaryHousing 노드: {len(housing_nodes)} 개")

# 3. WaterFacility 노드
water_nodes = []
if 'water_facility_clean' in data:
    for _, row in data['water_facility_clean'].iterrows():
        if pd.notna(row.get('lon')) and pd.notna(row.get('lat')):
            water_nodes.append({
                'id': f"water_{row['facility_id']}",
                'type': 'WaterFacility',
                'facility_id': str(row['facility_id']),
                'name': str(row.get('facility_name', '')),
                'sido': str(row.get('sido', '')),
                'sigungu': str(row.get('sigungu', '')),
                'address': str(row.get('address', '')),
                'lat': float(row['lat']),
                'lon': float(row['lon']),
                'status': str(row.get('status', ''))
            })

print(f"WaterFacility 노드: {len(water_nodes)} 개")

# 4. Admin 노드 (행정구역)
admin_set = set()
all_nodes = shelter_nodes + housing_nodes + water_nodes
for node in all_nodes:
    if 'sido' in node and 'sigungu' in node:
        admin_set.add((node['sido'], node['sigungu']))

admin_nodes = [
    {
        'id': f"admin_{sido.replace(' ', '_')}_{sigungu.replace(' ', '_')}",
        'type': 'Admin',
        'sido': sido,
        'sigungu': sigungu,
                'gu': sigungu,  # 그래프 스키마와 일치시키기 위해 gu 속성 추가
    }
    for sido, sigungu in admin_set
]

print(f"Admin 노드: {len(admin_nodes)} 개")

# 5. 관계 데이터 준비: (Shelter|TemporaryHousing|WaterFacility)-[:IN]->(Admin)
relationships = []

for node in shelter_nodes + housing_nodes + water_nodes:
    if 'sido' in node and 'sigungu' in node:
        admin_id = f"admin_{node['sido'].replace(' ', '_')}_{node['sigungu'].replace(' ', '_')}"
        relationships.append({
            'from_id': node['id'],
            'from_type': node['type'],
            'to_id': admin_id,
            'to_type': 'Admin',
            'relationship_type': 'IN'
        })

print(f"관계: {len(relationships)} 개")

# 노드와 관계를 DataFrame으로 저장 (CSV 출력용)
nodes_df = pd.DataFrame(shelter_nodes + housing_nodes + water_nodes + admin_nodes)
relationships_df = pd.DataFrame(relationships)

print(f"\n전체 노드: {len(nodes_df)} 개")
print(f"전체 관계: {len(relationships_df)} 개")
print(f"\n노드 타입별 통계:")
print(nodes_df['type'].value_counts())


Shelter 노드: 3136 개
TemporaryHousing 노드: 821 개
WaterFacility 노드: 0 개
Admin 노드: 25 개
관계: 3957 개

전체 노드: 3982 개
전체 관계: 3957 개

노드 타입별 통계:
type
Shelter             3136
TemporaryHousing     821
Admin                 25
Name: count, dtype: int64


## 7. 전처리된 데이터 저장


In [10]:
어

옥외대피소 CSV 저장: data/processed/outdoor_shelter_clean.csv
실내대피소 CSV 저장: data/processed/indoor_shelter_clean.csv
임시주거시설 CSV 저장: data/processed/temporary_housing_clean.csv
급수시설 CSV 저장: data/processed/water_facility_clean.csv

Neo4j 노드 저장: data/processed/neo4j_nodes.csv (3982 개)
Neo4j 관계 저장: data/processed/neo4j_relationships.csv (3957 개)
옥외대피소 GeoJSON 저장: data/processed/outdoor_shelter.geojson
실내대피소 GeoJSON 저장: data/processed/indoor_shelter.geojson
임시주거시설 GeoJSON 저장: data/processed/temporary_housing.geojson

전처리 완료!
